In [1]:
from sqlalchemy import create_engine
import pyodbc
import os
from urllib.parse import quote_plus
from dotenv import load_dotenv
import geopandas as gpd, pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# Load environment variables from .env
load_dotenv()

True

# Read

In [2]:
# --- Config ---
pg_config = {
    'username': 'analyst_ddl',
    'password': quote_plus(os.getenv('PG_PASSWORD')),
    'host': 'maritime-assets-db1-dev-geospatial.cluster-cinsmmsxwkgg.eu-west-1.rds.amazonaws.com',
    'port': 5432,
    'database': 'maritime_assets'
}

# --- Create PostgreSQL engine ---
pg_url = (
    f"postgresql://{pg_config['username']}:{pg_config['password']}@"
    f"{pg_config['host']}:{pg_config['port']}/{pg_config['database']}"
)
pg_engine = create_engine(pg_url)

In [3]:
# PG
q = f"""
select 	p.zone_id, 
		p.mt_id as matched_mt_port_id, 
		p.name as pg_name, 
		p.zone_type as pg_zone_type, 
		z.parent_zone_id,
		p.country_code as pg_country_code, 
	    ap.primary_related_zone_anch_id AS pg_related_zone_anch_id,
	    ap.primary_related_zone_port_id AS pg_related_zone_port_id,
	    coalesce(ec.terminal_count, 0) as pg_terminal_count, 
	    coalesce(ec.berth_count, 0) as pg_berth_count, 
	    coalesce(ec.berths_with_terminal_count, 0) as pg_berths_with_terminal_count, 
        coalesce(ps.stopped_sum, 0) as pg_stopped_unique_vessels_port,
        coalesce(bs.stopped_sum_berths, 0) as pg_stopped_unique_vessels_berths,
        p.polygon_geom as geometry
from sandbox.mview_master_ports p
left join ( --unique vessels stopped at port
    select p2.zone_id, sum(svp.mmsi_total) as stopped_sum
    from sandbox.mview_master_ports p2
    join sandbox.svp_hex_res10_2023_2024 svp
      on st_intersects(svp.geom, p2.polygon_geom)
    group by p2.zone_id
) ps on ps.zone_id = p.zone_id
left join ( --unique vessels stopped at port's berths
    select d.port_id as zone_id, sum(d.mmsi_total) as stopped_sum_berths
    from (
        select distinct b.port_id, svp.hex_id, svp.mmsi_total -- one row per (port, hex)
        from sandbox.mview_master_berths b
        join sandbox.svp_hex_res10_2023_2024 svp
          on st_intersects(svp.geom, b.polygon_geom)
    ) d
    group by d.port_id
) bs on bs.zone_id = p.zone_id
left join sandbox.v_port_anchorage_primary ap on ap.zone_id = p.zone_id --primary related anch/port
left join geospatial.zones z on z.zone_id = p.zone_id -- for parent_zone_id
left join ( --terminal, berths, berths with terminal counts
	select 
		b.port_id as zone_id, 
		count(distinct(b.terminal_id)) as terminal_count, 
		count(*) as berth_count, 
		count(*) filter (where b.terminal_id is not null) as berths_with_terminal_count
	from sandbox.mview_master_berths b
	group by b.port_id
) ec on ec.zone_id = p.zone_id
"""
gdf_pg = gpd.read_postgis(
    sql=q,
    con=pg_engine,
    geom_col='geometry'
)

In [4]:
# MT
q = f"""
select 
	p."PORT_ID" as mt_port_id, 
	p."PORT_NAME" as mt_port_name,
	p."PORT_TYPE" as mt_port_type,
	p."ENABLE_CALLS" as mt_enable_calls,
    coalesce(ec.mt_terminal_count, 0) as mt_terminal_count, 
    coalesce(ec.mt_berth_count, 0) as mt_berth_count, 
    coalesce(ec.mt_berths_with_terminal_count, 0) as mt_berths_with_terminal_count, 
    coalesce(ec.mt_berths_not_matched, 0) as mt_berths_not_matched,
	p.geometry 
from sandbox."MT_Ports" p 
left join (
	select 
		b."PORT_ID" as mt_port_id,
		count(distinct(b."TERMINAL_ID")) as mt_terminal_count, 
		count(*) as mt_berth_count, 
		count(*) filter (where b."TERMINAL_ID" is not null) as mt_berths_with_terminal_count,
		count(nm.not_matched) as mt_berths_not_matched
	from sandbox."MT_Berths" b
	left join (
		select mt_id as berth_id, 1 as not_matched
		from sandbox.mt_matching_not_matched nm 
		where nm.mt_type = 'berth'
	) nm on nm.berth_id = b."BERTH_ID" 
	group by b."PORT_ID"
) ec on p."PORT_ID" = ec.mt_port_id
where p."MOVING_SHIP_ID" is null
"""
gdf_mt = gpd.read_postgis(
    sql=q,
    con=pg_engine,
    geom_col='geometry'
)

In [5]:
# dbprim03 logs
q = f"""
select 
	zone_id, 
	mt_id as dbprim03_port_id,
	concat('https://mtapp-pr1661-uat.marinetraffic.xyz/en/ais/details/ports/',mt_id) as dbprim03_link,
	date(mtl."timestamp") as dbprim03_date, 
	mtl."statement" as dbprim03_statement
from sandbox.mt_transfer_log mtl 
where mt_table = 'ports'
and mtl.target_instance = 'dbprim03'
"""
df_log_dbprim03 = pd.read_sql(
    sql=q,
    con=pg_engine
)

In [6]:
# Prod logs
q = f"""
select 
	zone_id, 
	mt_id as prod_port_id,
	concat('https://www.marinetraffic.com/en/ais/details/ports/',mt_id) as prod_link,
	date(mtl."timestamp") as prod_date, 
	mtl."statement" as prod_statement
from sandbox.mt_transfer_log mtl 
where mt_table = 'ports'
and mtl.target_instance = 'dbprim'
"""
df_log_prod = pd.read_sql(
    sql=q,
    con=pg_engine
)

# Fix

In [7]:
# fix dtypes 
df_log_dbprim03['zone_id'] = df_log_dbprim03['zone_id'].astype('Int64')
df_log_dbprim03['dbprim03_port_id'] = df_log_dbprim03['dbprim03_port_id'].astype('Int64')

df_log_prod['zone_id'] = df_log_prod['zone_id'].astype('Int64')
df_log_prod['prod_port_id'] = df_log_prod['prod_port_id'].astype('Int64')

gdf_pg['matched_mt_port_id'] = gdf_pg['matched_mt_port_id'].astype('Int64')
gdf_pg['zone_id'] = gdf_pg['zone_id'].astype('Int64')

gdf_mt['mt_port_id'] = gdf_mt['mt_port_id'].astype('Int64')

In [8]:
# add matched zone_id in mt ports 
gdf_mt = gdf_mt.merge(gdf_pg[['matched_mt_port_id','zone_id']].rename(columns={'matched_mt_port_id':'mt_port_id', 'zone_id':'matched_zone_id'}), on='mt_port_id', how='left')

# Merge

In [9]:
# Initial Merge
merged_df = gdf_pg.merge(gdf_mt.rename(columns={'mt_port_id':'matched_mt_port_id'}), on='matched_mt_port_id', how='left').copy()
merged_df = merged_df.merge(df_log_dbprim03, on='zone_id', how='left')
merged_df = merged_df.merge(df_log_prod, on='zone_id', how='left')

In [10]:
# union of within / contains / overlaps (all area intersections, no boundary-only touches)
dfs = []
for pred in ["within", "contains", "overlaps"]:
    tmp = gpd.sjoin(
        gdf_pg[["zone_id", "matched_mt_port_id", "geometry"]],
        gdf_mt[["mt_port_id", "matched_zone_id", "mt_enable_calls", "geometry"]],
        how="inner",
        predicate=pred
    )[["zone_id", "matched_mt_port_id", "mt_port_id", "matched_zone_id", "mt_enable_calls"]]
    dfs.append(tmp)

area_intersections = pd.concat(dfs, ignore_index=True).drop_duplicates()

In [11]:
# add list of matched zone_ids of intersecting ports, matched excluded

filtered_intersections = area_intersections[area_intersections['matched_mt_port_id']!=area_intersections['mt_port_id']]

# aggregate matched_zone_id per zone_id into unique, sorted lists
df_intersections = (
    filtered_intersections.groupby("zone_id", as_index=False)
        .agg(zone_ids_of_intersecting_mt_ports=("matched_zone_id", lambda s: sorted({int(x) for x in s.dropna()})))  # unique ints
)

# add to merged
merged_df = merged_df.merge(df_intersections, on="zone_id", how="left")

# replace empty lists with NaN
merged_df["zone_ids_of_intersecting_mt_ports"] = (merged_df["zone_ids_of_intersecting_mt_ports"].apply(lambda v: np.nan if isinstance(v, list) and len(v) == 0 else v))

In [12]:
# add list of intersecting mt ports not matched with any zone_id, enable_calls=0 excluded

cond1 = area_intersections['matched_zone_id'].isna()
cond2 = area_intersections['mt_enable_calls']==True
filtered_intersections = area_intersections[cond1&cond2]

# aggregate mt_port_id per zone_id into unique, sorted lists
df_intersections = (
    filtered_intersections.groupby("zone_id", as_index=False)
        .agg(port_ids_of_intersecting_mt_not_matched=("mt_port_id", lambda s: sorted({int(x) for x in s.dropna()})))  # unique ints
)

merged_df = merged_df.merge(df_intersections, on="zone_id", how="left")

In [13]:
# Other calculated fields
merged_df['berth_diff'] = merged_df['pg_berth_count'] - merged_df['mt_berth_count'].fillna(0)
merged_df['terminal_diff'] = merged_df['pg_terminal_count'] - merged_df['mt_terminal_count'].fillna(0)
merged_df['berth_with_terminal_diff'] = merged_df['pg_berths_with_terminal_count'] - merged_df['mt_berths_with_terminal_count'].fillna(0)
merged_df['berth_stop_ratio'] = merged_df['pg_stopped_unique_vessels_berths']/merged_df['pg_stopped_unique_vessels_port']

In [14]:
# cols to keep
cols = ['zone_id', 'matched_mt_port_id', 'pg_name', 'pg_zone_type', 'parent_zone_id', 'pg_country_code',
       'pg_related_zone_anch_id', 'pg_related_zone_port_id',
       'pg_berth_count', 'berth_diff', 'pg_berths_with_terminal_count', 'berth_with_terminal_diff',
       'pg_stopped_unique_vessels_port', 'berth_stop_ratio',
       'mt_port_type', 'mt_enable_calls', 'mt_berths_not_matched',
       'zone_ids_of_intersecting_mt_ports', 'port_ids_of_intersecting_mt_not_matched',
       'dbprim03_port_id', 'dbprim03_link', 'dbprim03_date', 'dbprim03_statement',
       'prod_port_id', 'prod_link', 'prod_date', 'prod_statement']

merged_df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26334 entries, 0 to 26333
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   zone_id                                  26334 non-null  Int64  
 1   matched_mt_port_id                       19986 non-null  Int64  
 2   pg_name                                  26334 non-null  object 
 3   pg_zone_type                             26334 non-null  object 
 4   parent_zone_id                           3015 non-null   float64
 5   pg_country_code                          26080 non-null  object 
 6   pg_related_zone_anch_id                  2738 non-null   float64
 7   pg_related_zone_port_id                  2928 non-null   float64
 8   pg_berth_count                           26334 non-null  int64  
 9   berth_diff                               26334 non-null  float64
 10  pg_berths_with_terminal_count            26334

In [15]:
merged_df[cols].to_excel('port_list.xlsx', index=False)

# Filters

In [16]:
cond1 = merged_df['parent_zone_id'].isna()
cond2 = merged_df['port_ids_of_intersecting_mt_not_matched'].notna()
cond3 = merged_df['mt_enable_calls']==True

merged_df[cond1&cond2&cond3].info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 53 entries, 135 to 23887
Data columns (total 37 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   zone_id                                  53 non-null     Int64   
 1   matched_mt_port_id                       53 non-null     Int64   
 2   pg_name                                  53 non-null     object  
 3   pg_zone_type                             53 non-null     object  
 4   parent_zone_id                           0 non-null      float64 
 5   pg_country_code                          53 non-null     object  
 6   pg_related_zone_anch_id                  23 non-null     float64 
 7   pg_related_zone_port_id                  1 non-null      float64 
 8   pg_terminal_count                        53 non-null     int64   
 9   pg_berth_count                           53 non-null     int64   
 10  pg_berths_with_terminal_count   

In [17]:
cond1 = merged_df['zone_ids_of_intersecting_mt_ports'].notna()
cond1 = merged_df['zone_ids_of_intersecting_mt_ports'] \
    .apply(lambda v: len(v) if isinstance(v, list) else -1) == 0

merged_df[cond1]

,zone_id,matched_mt_port_id,pg_name,pg_zone_type,parent_zone_id,pg_country_code,pg_related_zone_anch_id,pg_related_zone_port_id,pg_terminal_count,pg_berth_count,...,prod_port_id,prod_link,prod_date,prod_statement,zone_ids_of_intersecting_mt_ports,port_ids_of_intersecting_mt_not_matched,berth_diff,terminal_diff,berth_with_terminal_diff,berth_stop_ratio


In [18]:
merged_df[cond1&cond2&cond3]

,zone_id,matched_mt_port_id,pg_name,pg_zone_type,parent_zone_id,pg_country_code,pg_related_zone_anch_id,pg_related_zone_port_id,pg_terminal_count,pg_berth_count,...,prod_port_id,prod_link,prod_date,prod_statement,zone_ids_of_intersecting_mt_ports,port_ids_of_intersecting_mt_not_matched,berth_diff,terminal_diff,berth_with_terminal_diff,berth_stop_ratio
